<center><font size='5'>Sorting</font></center>
<center><font size='3'>Eric Martin, CSE, UNSW</font></center>
<center><font size='3'>COMP9021 Principles of Programming</font></center>

In [ ]:
# Does not need to be executed if
# ~/.ipython/profile_default/ipython_config.py
# exists and contains:
# c.InteractiveShell.ast_node_interactivity = 'all'

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [ ]:
import zipfile
with zipfile.ZipFile('Illustrations.zip') as illustrations:
    illustrations.extractall('.')

In [ ]:
import ipywidgets as widgets
from copy import deepcopy
from neotermcolor import colored
from random import shuffle
import neotermcolor
neotermcolor.tty_aware = False

In [ ]:
class TraceSorting:
    def __init__(self, sorting_algorithm, nb_of_steps, batchersort=False):
        self.sorting_algorithm = sorting_algorithm
        self.nb_of_steps = nb_of_steps
        self.list = list(range(1, 9)) if batchersort else list(range(1, 11))
        self.tracer = sorting_algorithm(self.list)
        self.play = widgets.Play(interval=500, show_repeat=False)
        self.play.max = self.nb_of_steps(list(self.list))
        self.traces = widgets.Output()
        self.displayed_list = widgets.Text(self.display_list(),
                                           description='List to sort:'
                                          )
        self.default_order = widgets.Dropdown(options=['Sorted','Reversed',
                                                       'Random'
                                                      ]
                                             )
        self.length = widgets.Dropdown(options=[2, 4, 8, 16],
                                       value=8,
                                       description='List size',
                                      ) if batchersort\
                      else widgets.IntSlider(value=10, min=2, max=16,
                                             description='List size'
                                            )
        self.pause = widgets.FloatSlider(value=1, min=0.5, max=5, step=.5,
                                         description='Pause (s.)'
                                        )
        self.tracing_tab = widgets.VBox([self.play, self.traces])
        self.setting_tab = widgets.HBox([widgets.VBox([self.length,
                                                       self.pause
                                                      ]
                                                     ),
                                         widgets.VBox([self.default_order,
                                                       self.displayed_list
                                                      ]
                                                     )
                                        ]
                                       )
        self.tabs = widgets.Tab([self.tracing_tab, self.setting_tab])
        self.tabs.set_title(0, 'Trace algorithm')
        self.tabs.set_title(1, 'Settings')
        self.pause.observe(self.choose_waiting_time, 'value')
        self.length.observe(self.choose_list, 'value')
        self.default_order.observe(self.choose_list, 'value')
        self.play.observe(self.trace, 'value')

    def reinitialise_tracer(self):
        self.play.max = self.nb_of_steps(list(self.list))
        self.tracer.close()
        self.tracer = self.sorting_algorithm(list(self.list))

    def display_list(self):
        return ' '.join(str(e) for e in self.list)

    def trace(self, _):
        with self.traces:
            if not self.play.value:
                print()
            else:
                next(self.tracer)
            if not self.play.value or self.play.value == self.play.max:
                self.reinitialise_tracer()

    def choose_waiting_time(self, _):
        self.play.interval = self.pause.value * 1000

    def choose_list(self, _):
        self.list = list(range(1, self.length.value + 1))
        if self.default_order.value == 'Reversed':
            self.list.reverse()
        elif self.default_order.value == 'Random':
            while True:
                shuffle(self.list)
                if self.length.value == 2\
                or self.list != sorted(range(1, self.length.value + 1))\
                   and self.list !=\
                              list(reversed(range(1, self.length.value + 1))):
                    break
        self.displayed_list.value = self.display_list()
        self.play.value = 0
        self.reinitialise_tracer()

In [ ]:
def initialise_colouring(L, blue=True):
    return {i: blue and {'color': 'blue'} or {} for i in range(len(L))}

def display(L, item_colours, last_in_line=True):
    print(' '.join(colored(L[i], **item_colours[i]) for i in range(len(L))
                  ), end=last_in_line and '\n' or '    '
         )

def display_2(L1, item_colours_1, L2, item_colours_2):
    display(L1, item_colours_1, False)
    display(L2, item_colours_2)

# Preliminaries

Both the standalone widget and this Jupyter sheet's ipywidgets make use of boldface and colours as follows.

* **Boldface:** When two numbers are being compared, they become boldface.
* **Blue:** Blue is the colour of the "live" numbers. For iterative algorithms, all numbers are blue unless their current status is special and they receive a different colour. For recursive algorithms (Quicksort, Mergesort), the numbers in blue (together with or excepted those whose status is special and which receive a different colour) are those involved in the current recursive call.
* **Red:** For all algorithms except Mergesort, two numbers are coloured red when they are about to be swapped. For Mergesort, a number is coloured in red when it is about to be copied to a location indicated by a star.
* **Yellow:** Yellow is used
    * when a number is being compared with another number (so its font is boldface), will not be swapped, and is not known to be in place,
    * or when a number has been copied from somewhere else (which except for Mergesort, means that it has been swapped with another number) and is not known to have been put in place.
* **Green:** Green is the colour of the numbers that have been put in place. For Insertionsort and Shellsort, this happens at the very end. For Bubblesort, this can happen earlier. For all other algorithms, this has to happen earlier for some numbers.

The list to sort is denoted $L$; its length, $N$. It is assumed that the list consists of the integers 1, ..., $N$. When computing numbers of comparisons, the trivial case $N=0$ is usually ignored and $N$ is tacitly assumed to be strictly positive.

# Bubble sort

## Description

In a first round of comparisons, $L$'s first and second elements are compared, swapped if they are misordered, $L$'s (possibly newly) second and third elements are compared, swapped if they are misordered, etc.

* Eventually, $N$ ends up at the end of $L$, either because it was there to start with and there is no swap at the end of this round, or because it has been successively swapped with all elements to its right.
* If to start with, $L$ does end in $N$, then $N-1$ ends up at $L$'s penultimate position, either because it was there to start with and this round ends with no swap associated with the last 2 comparisons, or because it has been successively swapped with all elements to its right except $N$.
* If to start with, $L$ does end in $N-1$, $N$, then $N-2$ ends up at $L$'s third last position, either because it was there to start with and this round ends with no swap associated with the last 3 comparisons, or because it has been successively swapped with all elements to its right except $N-1$ and $N$.
* ...

If all of $L$'s elements are ordered to start with, then no swap has happened, all elements stay in place, and the procedure stops. If $i\in\{0,\dots,N-2\}$ is largest such that the element at index $i$ has been swapped with the element at index $i+1$, then $L$ is sure to have $i+2$, ..., $N$ properly placed at the end of the first round, and a second round takes place that involves $L$'s $i+1$ first elements and proceeds identically. If there is no swap in this round, then all elements were ordered when the first round ended, all elements stay in place, and the procedure stops. Otherwise, a third round starts, working on $L$'s first $j+1$ elements where $j\in\{0,\dots,i-1\}$ is largest such that the element at index $j$ has been swapped with the element at index $j+1$ during the second round.

After at most $N-1$ rounds, the procedure stops with all of all $L$'s elements in place.

## Implementation

In [ ]:
def bubble_sort(L):
    bound = len(L) - 1
    swapped = True
    while swapped and bound:
        swapped = False
        for i in range(bound):
            if L[i] > L[i + 1]:
                L[i], L[i + 1] = L[i + 1], L[i]
                swapped = True
                bound = i

In [ ]:
L = list(range(1, 10))
shuffle(L)
L
bubble_sort(L)
L

## Simulation

In [ ]:
def bubble_sort_nb_of_steps(L):
    nb_of_steps = 1
    bound = len(L) - 1
    swapped = True
    while swapped and bound:
        swapped = False
        new_bound = 0
        for i in range(bound):
            if L[i] > L[i + 1]:
                nb_of_steps += 2
                L[i], L[i + 1] = L[i + 1], L[i]
                swapped = True
                new_bound = i
            else:
                nb_of_steps += 1 
                if i + 1 == bound:
                    nb_of_steps += 1
        bound = new_bound
    return nb_of_steps

def trace_bubble_sort(L):
    item_colours = initialise_colouring(L)
    display(L, item_colours)
    yield
    bound = len(L) - 1
    swapped = True
    while swapped and bound:
        swapped = False
        new_bound = 0
        for i in range(bound):
            item_colours[i]['attrs'] = item_colours[i + 1]['attrs'] = 'bold'
            if L[i] > L[i + 1]:
                item_colours[i]['color'] = item_colours[i + 1]['color'] = 'red'
                display(L, item_colours)
                yield
                item_colours[i]['attrs'] = item_colours[i + 1]['attrs'] = ''
                L[i], L[i + 1] = L[i + 1], L[i]
                if bound == 1 and i == 0:
                    item_colours[i]['color'] = 'green'
                else:
                    item_colours[i]['color'] = 'yellow'
                if i + 1 == bound:
                    item_colours[i + 1]['color'] = 'green'
                else:
                    item_colours[i + 1]['color'] = 'yellow'
                swapped = True
                new_bound = i
                display(L, item_colours)
                yield
                if bound != 1 or i:
                    item_colours[i]['color'] ='blue'
            else:
                item_colours[i]['color'] = item_colours[i + 1]['color'] =\
                                                                       'yellow'
                display(L, item_colours)
                yield
                item_colours[i]['attrs'] = item_colours[i + 1]['attrs'] = ''
                item_colours[i]['color'] = 'blue'
                if i + 1 == bound:                
                    for j in range(new_bound + 1, bound + 1):
                        item_colours[j]['color'] = 'green'
                    if new_bound == 0:
                        item_colours[0]['color'] = 'green'
                    display(L, item_colours)
                    yield
        bound = new_bound

In [ ]:
widgets.VBox([TraceSorting(trace_bubble_sort, bubble_sort_nb_of_steps).tabs])

## Number of comparisons in the best case

When $L$ is sorted to start with, there is a unique round of $N-1$ comparisons.

## Number of comparisons in the worst case

When $L$ is in reverse order to start with, there is a round of $N-1$ comparisons, followed by a round of $N-2$ comparisons, ..., and eventually an $(N-1)^{\mathrm{th}}$ round of 1 comparison, for a total of $\frac{N(N-1)}{2}$ comparisons.

# Selection sort

## Description

In a first round of comparisons, $L$'s first and second elements are compared and the index of the smallest of the two, say element $e_1$, is recorded, then $e_1$ is compared with $L$'s third element and the index of the smallest of the two, say element $e_2$, is recorded, then $e_2$ is compared with $L$'s fourth element and the index of the smallest of the two is recorded... Eventually, the index $i$ of $L$'s smallest element is known. In case $i$ is not equal to 0 and so $L$'s smallest element is not $L$'s first element, then both elements are swapped, resulting in $L$'s smallest element to be in place; othwerwise, the smallest element is found out to have been in place to start with.

A second round of comparisons takes place with all of $L$'s elements except the first one that proceeds identically, that results in $L$'s second element to be put or remain in place.

After exactly $N-1$ rounds, the procedure stops with all of all $L$'s elements in place.

In [ ]:
def selection_sort(L):
    for i in range(len(L) - 1):
        index_of_min = i
        for j in range(i + 1, len(L)):
            if L[j] < L[index_of_min]:
                index_of_min = j
        if index_of_min != i:
            L[i], L[index_of_min] = L[index_of_min], L[i]

## Implementation

In [ ]:
L = list(range(1, 10))
shuffle(L)
L
selection_sort(L)
L

## Simulation

In [ ]:
def selection_sort_nb_of_steps(L):
    nb_of_steps = 1
    for i in range(len(L) - 1):
        index_of_min = i
        for j in range(i + 1, len(L)):
            nb_of_steps += 1
            if L[j] < L[index_of_min]:
                index_of_min = j
        if index_of_min != i:
            nb_of_steps += 2
            L[i], L[index_of_min] = L[index_of_min], L[i]
        else:
            nb_of_steps += 1
    return nb_of_steps

def trace_selection_sort(L):
    item_colours = initialise_colouring(L)
    display(L, item_colours)
    yield
    for i in range(len(L) - 1):
        index_of_min = i
        item_colours[index_of_min]['color'] = 'yellow'
        item_colours[index_of_min]['attrs'] = 'bold'
        for j in range(i + 1, len(L)):
            item_colours[j]['color'] = 'yellow'
            item_colours[j]['attrs'] = 'bold'
            display(L, item_colours)
            yield
            if L[j] < L[index_of_min]:
                item_colours[index_of_min]['color'] = 'blue'
                item_colours[index_of_min]['attrs'] = ''
                index_of_min = j
            else:
                item_colours[j]['color'] = 'blue'
                item_colours[j]['attrs'] = ''
        item_colours[index_of_min]['attrs'] = ''
        if index_of_min != i:
            item_colours[index_of_min]['color'] = item_colours[i]['color'] =\
                                                                          'red'
            display(L, item_colours)
            yield
            L[i], L[index_of_min] = L[index_of_min], L[i]
            item_colours[i]['color'] = 'green'
            if i == len(L) - 2 or len(L) == 2:
                item_colours[index_of_min]['color'] = 'green'
                display(L, item_colours)
                yield
            else:
                item_colours[index_of_min]['color'] = 'yellow'
                display(L, item_colours)
                yield
                item_colours[index_of_min]['color'] = 'blue'
        else:
            item_colours[index_of_min]['color'] = 'green'
            if i == len(L) - 2 or len(L) == 2:
                item_colours[i + 1]['color'] = 'green'
            display(L, item_colours)
            yield

In [ ]:
widgets.VBox([TraceSorting(trace_selection_sort,
                           selection_sort_nb_of_steps
                          ).tabs
             ]
            )

## Number of comparisons in all cases

In any case, there is a round of $N-1$ comparisons, followed by a round of $N-2$ comparisons, ..., and eventually an $(N-1)^{\mathrm{th}}$ round of 1 comparison, for a total of $\frac{N(N-1)}{2}$ comparisons.

# Insertion sort

## Description

* In a first round of comparisons, $L$'s first and second elements are compared; both are swapped if they are misordered. So $L$'s first two elements end up being ordered.
* In a second round of comparisons, $L$'s third element, say $e$, and $L$'s currently second element are compared; both are swapped if they are misordered in which case $e$ is compared with $L$'s currently first element; both are swapped if they are misordered. So $L$'s first three elements end up being ordered.
* In a third round of comparisons, $L$'s fourth element, say $e$, and $L$'s currently third element are compared; both are swapped if they are misordered in which case $e$ is compared with $L$'s currently second element; both are swapped if they are misordered in which case $e$ is compared with $L$'s currently first element; both are swapped if they are misordered. So $L$'s first four elements end up being ordered.
* ...

After exactly $N-1$ rounds, the procedure stops with all of all $L$'s elements in place.

## Implementation

In [ ]:
def insertion_sort(L):
    for i in range(1, len(L)):
        j = i
        while j and L[j - 1] > L[j]:
            L[j - 1], L[j] = L[j], L[j - 1]
            j -= 1

In [ ]:
L = list(range(1, 10))
shuffle(L)
L
insertion_sort(L)
L

## Simulation

In [ ]:
def insertion_sort_nb_of_steps(L):
    nb_of_steps = 1
    for i in range(1, len(L)):
        j = i
        while j and L[j - 1] > L[j]:
            nb_of_steps += 1
            L[j - 1], L[j] = L[j], L[j - 1]
            if j != 1 or i != len(L) - 1:
                nb_of_steps += 1
            j -= 1
        if j:
            nb_of_steps += 1
        elif i == len(L) - 1:
            return nb_of_steps + 1
    return nb_of_steps + 1

def trace_insertion_sort(L):
    item_colours = initialise_colouring(L)
    display(L, item_colours)
    yield
    for i in range(1, len(L)):
        j = i
        while j and L[j - 1] > L[j]:
            item_colours[j - 1]['color'] = item_colours[j]['color'] = 'red'
            item_colours[j - 1]['attrs'] = item_colours[j]['attrs'] = 'bold'
            display(L, item_colours)
            yield
            item_colours[j - 1]['attrs'] = item_colours[j]['attrs'] = ''
            L[j - 1], L[j] = L[j], L[j - 1]
            if j != 1 or i != len(L) - 1:
                item_colours[j - 1]['color'] = item_colours[j]['color'] =\
                                                                       'yellow'
                display(L, item_colours)
                yield
                item_colours[j]['color'] = 'blue'
            j -= 1
        if j:
            item_colours[j - 1]['color'] = item_colours[j]['color'] = 'yellow'
            item_colours[j - 1]['attrs'] = item_colours[j]['attrs'] = 'bold'
            display(L, item_colours)
            yield
            item_colours[j - 1]['color'] = item_colours[j]['color'] = 'blue'
            item_colours[j - 1]['attrs'] = item_colours[j]['attrs'] = ''
        elif i == len(L) - 1:
            for i in range(len(L)):
                item_colours[i]['color'] = 'green'
            display(L, item_colours)
            yield
            return
        else:
            item_colours[j]['color'] = 'blue'
    for i in range(len(L)):
        item_colours[i]['color'] = 'green'
    display(L, item_colours)
    yield

In [ ]:
widgets.VBox([TraceSorting(trace_insertion_sort,
                           insertion_sort_nb_of_steps
                          ).tabs
             ]
            )

## Number of comparisons in the best case

When $L$ is sorted to start with, there is a unique comparison for each of the $N-1$ rounds, for a total of $N-1$ comparisons.

## Number of comparisons in the worst case

When $L$ is in reverse order to start with, there is a round of 1 comparison, followed by a round of 2 comparisons, ..., and eventually an $(N-1)^{\mathrm{th}}$ round of $N-1$ comparisons, for a total of $\frac{N(N-1)}{2}$ comparisons.

# Shell sort

## Description

Choose strictly positive integers $k$, $h_1$, $h_2$, ..., $h_k$ such that $1=h_1<h_2<\dots<h_k<N$. There are $k$ turns of comparisons, one associated with $h_k$, then one associated with $h_{k-1}$, ..., and eventually one associated with $h_1$. Each of these turns, associated with, say, $h$, consists of at least one and at most $h$ series of comparisons, each of which consists of the rounds of comparisons described for Insertion sort and applied to:

* $L$'s elements of index 0, $h$, $2h$, $3h$, ..., $\llcorner\frac{N-1}{h}\lrcorner h$ for the first series;
* in case $h<N-1$, $L$'s elements of index 1, $1+h$, $1+2h$, $1+3h$, ..., $1+\llcorner\frac{N-2}{h}\lrcorner h$ for the second series;
* in case $h<N-2$, $L$'s elements of index 2, $2+h$, $2+2h$, $2+3h$, ..., $2+\llcorner\frac{N-3}{h}\lrcorner h$ for the third series;
* ...
* in case $h<N-h+1$, $L$'s elements of index $h-1$, $(h-1)+h$, $(h-1)+2h$, $(h-1)+3h$, ..., $h-1+\llcorner\frac{N-h}{h}\lrcorner h$ for the $h^{\mathrm{th}}$ series.

For instance, if $N=19$ and $h=5$, there are 5 series of comparisons, each of which is associated with the corresponding line below and operates on those of $L$'s elements that are indicated with a star:

        *----*----*----*---
        -*----*----*----*--
        --*----*----*----*-
        ---*----*----*----*
        ----*----*----*----

Since $h_1=1$, the $k^{th}$ turn of comparisons consists of a single series of comparisons and is Insertion sort itself, applied to what $L$ is when this last turn is about to start, expectedly close to 1, ...., $n$ thanks to the comparisons and possible swaps that have been performed during the previous $k-1$ turns. 

Implementation, simulation and analysis are based on Pratt's method, where $a_1$, $a_2$, ..., $a_k$ is an initial segment of 1, 2, 3, 4, 6, 8, 9, 12, 16, 18... that is, the ordered sequence of all products of powers of 2 and powers of 3 smaller than $N$. For this particular choice, a round can be reduced to comparing and possibly swapping the first two pairs of integers that in standard Insertion sort, are considered at the beginning of the round. This is illustrated below for the last round of each of the 5 series: for each line, the elements of the list that are indicated by the stars are compared and possibly swapped, but it is not necessary to then compare the smaller of the two, say $x$, with the element of the list indicated by the last dot on the line, say $y$, as $y$ is guaranteed to be smaller than $x$, as will be shown in Section [Proof of correctness and number of comparisons in all cases](#Proof-of-correctness-and-number-of-comparisons-in-all-cases).

        .----.----*----*---
        -.----.----*----*--
        --.----.----*----*-
        ---.----.----*----*
        ----.----*----*----

As a consequence, rather than processing series after series, each series consisting of rounds where a single comparison is performed, which for this example can be illustrated as follows:

        *----*----.----.---
        .----*----*----.---
        .----.----*----*---
        -*----*----.----.--
        -.----*----*----.--
        -.----.----*----*--
        --*----*----.----.-
        --.----*----*----.-
        --.----.----*----*-
        ---*----*----.----.
        ---.----*----*----.
        ---.----.----*----*
        ----*----*----.----
        ----.----*----*----

it is simpler to entwine the series and just process the rounds as follows:

        *----*----.----.---
        -*----*----.----.--
        --*----*----.----.-
        ---*----*----.----.
        ----*----*----.----
        .----*----*----.---
        -.----*----*----.--
        --.----*----*----.-
        ---.----*----*----.
        ----.----*----*----
        .----.----*----*---
        -.----.----*----*--
        --.----.----*----*-
        ---.----.----*----*

## Implementation

In [ ]:
def shell_sort(L):
    for h in range(len(L) - 1, 0, -1):
        # We use Pratt's method which uses as gaps all numbers of the form
        # 2^i * 3^j
        p = h
        while p % 2 == 0:
            p //= 2
        while p % 3 == 0:
            p //= 3
        if p != 1:
            continue
        for i in range(len(L) - h):
            if L[i + h] < L[i]:
                L[i + h], L[i] = L[i], L[i + h]

In [ ]:
L = list(range(1, 10))
shuffle(L)
L
shell_sort(L)
L

## Simulation

In [ ]:
def shell_sort_nb_of_steps(L):
    nb_of_steps = 1
    for h in range(len(L) - 1, 0, -1):
        p = h
        while p % 2 == 0:
            p //= 2
        while p % 3 == 0:
            p //= 3
        if p != 1:
            continue
        for i in range(len(L) - h):
            nb_of_steps += 1
            if L[i + h] < L[i]:
                nb_of_steps += 1
                L[i + h], L[i] = L[i], L[i + h]
                if h == 1 and i == len(L) - h - 1:
                    return nb_of_steps
    return nb_of_steps + 1

def trace_shell_sort(L):
    item_colours = initialise_colouring(L)
    display(L, item_colours)
    yield
    for h in range(len(L) - 1, 0, -1):
        p = h
        while p % 2 == 0:
            p //= 2
        while p % 3 == 0:
            p //= 3
        if p != 1:
            continue
        for i in range(len(L) - h):
            item_colours[i]['attrs'] = item_colours[i + h]['attrs'] = 'bold'
            if L[i + h] < L[i]:
                item_colours[i]['color'] = item_colours[i + h]['color'] = 'red'
                display(L, item_colours)
                yield
                L[i + h], L[i] = L[i], L[i + h]
                item_colours[i]['attrs'] = item_colours[i + h]['attrs'] = ''
                item_colours[i]['color'] = 'green' if h == 1 else 'yellow'
                item_colours[i + h]['color'] = 'green'\
                    if h == 1 and i == len(L) - h - 1 else 'yellow'
                display(L, item_colours)
                yield
                if h == 1 and i == len(L) - h - 1:
                    return
            else:
                item_colours[i]['color'] = item_colours[i + h]['color'] =\
                                                                       'yellow'
                display(L, item_colours)
                yield
                item_colours[i]['attrs'] = item_colours[i + h]['attrs'] = ''
            if h != 1:
                item_colours[i]['color'] = item_colours[i + h]['color'] =\
                                                                         'blue'
            else:
                item_colours[i]['color'] = 'green'
                item_colours[i + h]['color'] = 'green'\
                    if i == len(L) - h - 1 else 'yellow'
            if h == 1 and i == len(L) - h - 1:
                display(L, item_colours)
                yield

In [ ]:
widgets.VBox([TraceSorting(trace_shell_sort, shell_sort_nb_of_steps).tabs])

## Proof of correctness and number of comparisons in all cases

We prove that Pratt's method is correct and exactly $\Sigma_{i,j\in\mathbb N,\,2^i3^j<N}(N-2^i3^j)$ comparisons are performed. Since there are at most $\ulcorner\log_2(N)\urcorner$ powers of 2 smaller than $N$ and at most $\ulcorner\log_3(N)\urcorner$ powers of 3 smaller than $N$, that is $O\bigl(N\log(N)^2\bigr)$ comparisons. The proof is based on the two propositions that follow, as we will see applying the second one to $a=2$ and $b=3$.

Given a natural number $h$ and a list $L$, say that $L$ is *$h$-sorted* if for all natural numbers $i$ smaller than the length of the list minus $h$, $L[i]\leq L[i+h]$. The `h_sort(L, h)` function below is designed to $h$-sort a list $L$. As described above, it performs at most $h$ series of comparison applied to:

* $L$'s elements of index 0, $h$, $2h$, $3h$, ..., $\llcorner\frac{N-1}{h}\lrcorner h$ for the first series;
* in case $h<N-1$, $L$'s elements of index 1, $1+h$, $1+2h$, $1+3h$, ..., $1+\llcorner\frac{N-2}{h}\lrcorner h$ for the second series;
* in case $h<N-2$, $L$'s elements of index 2, $2+h$, $2+2h$, $2+3h$, ..., $2+\llcorner\frac{N-3}{h}\lrcorner h$ for the third series;
* ...
* in case $h<N-h+1$, $L$'s elements of index $h-1$, $(h-1)+h$, $(h-1)+2h$, $(h-1)+3h$, ..., $h-1+\llcorner\frac{N-h}{h}\lrcorner h$ for the $h^{\mathrm{th}}$ series.

Each series consists of rounds of comparisons as done by Insertion sort, except that the round does not stop when the two elements being compared do not have to be swapped, but going all the way down the sequence of members of $L$ that are $h$-indexes apart, that is, all the way down to $L[0]$ for the first series, all the way down to $L[1]$ for the second series in case $h<N-1$, ..., all the way down to $L[h-1]$ for the $h^{\mathrm{th}}$ series in case $h<N-h+1$. Of course, all the extra comparisons are "for nothing" and the result is the same as if the rounds of comparisons were limited to those done by Insertion sort. The function `h_sort(L, h)` is introduced for the sake of stating and proving the first proposition below.

In [ ]:
def h_sort(L, h):
    for i in range(h, min(2 * h, len(L))):
        for j in range(i, len(L), h):
            for k in range(j, h - 1, -h):
                if L[k - h] > L[k]:
                    L[k - h], L[k] = L[k], L[k - h]

In [ ]:
L = list(range(5, 10)) + list(range(5))
L
for h in range(10, 0, -1):
    print(f'{h}-sorting previous list')
    h_sort(L, h)
    L

**Proposition 1.**<a name=prop_1></a>$\ $ _Let natural numbers $N,h,h'$ and an $h$-sorted list $L$ of length $N$ be given. After `h_sort(L,h')` has been executed, $L$ is still $h$-sorted._

*Proof.*$\ $ Let $L_1$ be $L$ preceded by $\overbrace{-\infty,\dots,-\infty}^h$; let $L_2$ be $L$ followed by $\overbrace{\infty,\dots,\infty}^h$. Since $L$ is $h$-sorted, for all $i<N+h$, $L_1[i]\leq L_2[i]$.

        | -∞ | ... |  -∞  |L[0]| ... |L[N-1-h]|L[N-h]| ... |L[N-1]|
        -----------------------------------------------------------
        |L[0]| ... |L[h-1]|L[h]| ... | L[N-1] |  +∞  | ... |  +∞  |

Given $i,j<N+h$, every comparison possibly followed by a swap performed by `h_sort(L_1,h')` between $L_1[i]$ and $L_1[j]$ corresponds to a comparison possibly followed by a swap performed by `h_sort(L_2,h')` between $L_1[i]$ and $L_2[j]$, updating the current values of $L_1[i]$, $L_1[j]$, $L_2[i]$ and $L_2[j]$, in such a way that 

* $L_1[i]$ becomes equal to $\min\bigl(L_1[i],L_1[j]\bigr)$, which is at most equal to $\min\bigl(L_2[i],L_2[j]\bigr)$, which is what $L_2[i]$ becomes equal to;
* $L_2[j]$ becomes equal to $\max\bigl(L_1[i],L_1[j]\bigr)$, which is at most equal to $\max\bigl(L_2[i],L_2[j]\bigr)$, which is what $L_2[j]$ becomes equal to.

So after `h_sort(L_1,h')` and `h_sort(L_2,h')` have been executed, $L_1$ and $L_2$ have been changed in such a way that we still have $L_1[i]\leq L_2[i]$ for all $i<N+h$. Obviously, the first $h$ elements of $L_1$ stayed in place, and the last elements of $L_2$ stayed in place. So $L_1$ has been changed to $\overbrace{-\infty,\dots,-\infty}^h$ followed by what $L$ has become after executing `h_sort(L,h')`, and $L_2$ has been changed to what $L$ has become after executing `h_sort(L,h')` followed by $\overbrace{\infty,\dots,\infty}^h$. Therefore, after executing `h_sort(L,h')`, we still have $L[i]\leq L_2[i+h]$ for all $i<N-h$, as wanted.$\ $ Q.E.D.

The second proposition is a particular case of _Frobenius problem_.

**Proposition 2.**<a name=prop_2></a>$\ $ _Let $a$ and $b$ be two relatively prime integers greater than 1. There exists a larger integer $n$ for which there are no natural numbers $x$ and $y$ such that $n=xa+by$; moreover, $n=ab-a-b$._

*Proof.*$\ $ Without loss of generality, assume that $a<b$. Note that the $a$ integers $0b$, $1b$, $2b$, ..., $(a-1)b$ are pairwise distinct modulo $a$: otherwise, there would be distinct $i,j\in\{0,\dots,a-1\}$ such that $(i-j)b=0\bmod a$, which together with $\gcd(a,b)=1$ imples that $i-j=0\bmod a$, which is impossible because $0<i-j<a$.

Suppose for a contradiction that there exists natural numbers $x$ and $y$ such that $ab−a−b=xa+by$. Then $ab=(x+1)a+(y+1)b$, from which we derive both $(y+1)b=0\bmod a$ and $y+1<a$, in contraction with the previous observation (and the fact that $0b=0\bmod a$).

To complete the proof of the proposition, it suffices to verify that the next $a$ consecutive integers, namely, $ab−a−b+1$, $ab−a−b+2$, ..., $ab−a−b+a$ are all of the form $xa+yb$ for some natural numbers $x$ and $y$. That is straighforward for the last one: $ab−a−b+a=0a+(a−1)b$. The smallest of the other ones, namely, $ab−a−b+1$, is equal to $(a−1)b+1−a$, hence is strictly greater than $(a−1)b−b$, that is, strictly greater than $(a−2)b$, hence is strictly greater than all of $0a+0b$, $0a+1b$, ..., $0a+(a-2)b$. Hence each of $ab−a−b+1$, $ab−a−b+2$, ..., $ab−a−b+(a-1)$ has the same remainder modulo $a$ as one of the all smaller integers $0b$, $1b$, $2b$, ..., $(a-2)b$, so is equal to one of these integers plus some strictly positive multiple of $a$, hence is of the form $xa+yb$ for some natural numbers $x$ and $y$.$\ $ Q.E.D.

Let us now get to the proof that Pratt's method is correct and exactly $\Sigma_{i,j\in\mathbb N,\,2^i3^j<N}(N-2^i3^j)$ comparisons are performed. Consider $h\in\{1,2,3,4,6,8,9,12,16,18...\}$ and the time when the turn of comparisons associated with $h$ is about to start, so for all natural numbers $p$ and $q$ with $h<2^p3^q<N$, the turn of comparisons associated with $2^p3^q$ has been performed. Let $L'$ denote what $L$ is at that time. Let $i<h$ be given, and consider the round of comparisons that Insertion sort has to perform on the elements of the list at index $i$, $i+h$, $i+2h$, ..., $i+kh$ for some $k>1$, having performed all rounds of comparisons on the elements of list at index $i$, $i+h$, $i+2h$, ..., $i+k'$ successively for $k'$ equal to 1, then 2,... and finally $k-1$. So the elements of the list at index $i+kh$ and $i+(k-1)h$ are compared and swapped if needed. Suppose they are swapped. We have to verify the claim that the element of the list at index $i+(k-2)h$ is necessarily smaller than the element of the list at index $i+kh$ before the swap, hence there is no need to compare them and the round can stop. Indeed, the element of the list at index $i+kh$ is $L'[i+kh]$, and the element of the list at index $i+(k-2)h$ is $L'[i+k'h]$ for some $k'\in\{0,1,\dots,k-2\}$, depending on the swaps that took place in the previous rounds. By Proposition [2](#prop_2), because $2\times 3-2-3=1$, there exists natural numbers $x$ and $y$ such that $k-k'=x2+y3$. Write $h$ as $2^p3^q$. Then $i+kh-(i+k'h)=x2^{p+1}3^q+y2^p3^{q+1}$. By Proposition [1](#prop_1), $L'$ is both $2^{p+1}3^q$-sorted and $2^p3^{q+1}$-sorted (possibly trivially for values at least equal to $N$), hence

\begin{equation*}
L'[i+k'h]\leq L'[i+k'h+x2^{p+1}3^q]\leq L'[i+k'h+x2^{p+1}3^q+y2^p3^{q+1}]=L'[i+kh]
\end{equation*}

as wanted. Clearly, there are $N-h$ rounds of comparisons for the turn of comparisons associated with $h$, thereby completing the verification that Pratt's method is correct and performs the claimed number of comparisons.

# Merge sort

## Description

If $L$ has less than 2 elements then $L$ is sorted; otherwise, $L$ is split in two halves, say $L_1$ and $L_2$, of size $\llcorner\frac{N}{2}\lrcorner$ and $\ulcorner\frac{N}{2}\urcorner$, respectively (so $L_2$ has one more element than $L_1$ in case $N$ is odd), both of which are sorted recursively. At that point, $L$'s smallest element is either $L_1$'s first element or $L_2$'s first element.

* In the first case, $L$'s second smallest element is either $L_1$'s second element or $L_2$'s first element.
    * In the first case, $L$'s third smallest element is either $L_1$'s third element or $L_2$'s first element.
        * ...
    * In the second case, $L$'s third smallest element is either $L_1$'s second element or $L_2$'s second element.
        * ...
* In the second case, $L$'s second smallest element is either $L_1$'s first element or $L_2$'s second element.
    * In the first case, $L$'s third smallest element is either $L_1$'s second element or $L_2$'s second element.
        * ...
    * In the second case, $L$'s third smallest element is either $L_1$'s first element or $L_2$'s third element.
        * ...
        
Eventually, either all of $L_1$'s elements have been processed, in which case what remains unprocessed in $L_2$ is an ordered sequence of $L$'s largest elements, or all of $L_2$'s elements have been processed, in which case what remains unprocessed in $L_1$ is an ordered sequence of $L$'s largest elements.

The list $L$ itself and a copy of $L$, say $L'$, are sufficient to carry out all recursion calls. Indeed, to sort $L$, $L'$ two halves can be sorted and merged into $L$.

* To sort $L'$'s first half, say $L'_1$, $L$'s first 2 quarters can be sorted and merged into $L'_1$.
    * To sort $L$'s first quarter, say $L_{1,1}$, $L'$'s first 2 eighths can be sorted and merged into $L_{1,1}$.
        * ...
    * To sort $L$'s second quarter, say $L_{1,2}$, $L'$'s second 2 eighths can be sorted and merged into $L_{1,2}$.
        * ...
* To sort $L'$'s second half, say $L'_2$, $L$'s last 2 quarters can be sorted and merged into $L'_2$.
    * To sort $L$'s third quarter, say $L_{2,1}$, $L'$'s third 2 eighths can be sorted and merged into $L_{2,1}$.
        * ...
    * To sort $L$'s last quarter, say $L_{2,2}$, $L'$'s last 2 eighths can be sorted and merged into $L_{2,2}$.
        * ...

This is illustrated below for merging the two sorted halves of $L'$ into $L$ (arrows at the top), and the 4 sorted quarters of $L$ into the two halves of $L'$ (arrows at the bottom).

<div><img src="Illustrations/merge_sort_1.pdf" width="400"/></div>

## Implementation

In [ ]:
def merge_sort(L):
    L_copy = list(L)
    mergesort(L, L_copy, 0, len(L))

def mergesort(L1, L2, start, end):
    if end - start < 2:
        return
    half_length = start + (end - start) // 2
    mergesort(L2, L1, start, half_length)
    mergesort(L2, L1, half_length, end)
    i = start
    i1 = start
    i2 = half_length
    while i1 < half_length and i2 < end:
        if L2[i1] <= L2[i2]:
            L1[i] = L2[i1]
            i1 += 1
        else:
            L1[i] = L2[i2]
            i2 += 1
        i += 1
    while i1 < half_length:
        L1[i] = L2[i1]
        i1 += 1
        i += 1
    while i2 < end:
        L1[i] = L2[i2]
        i2 += 1
        i += 1

In [ ]:
L = list(range(1, 10))
shuffle(L)
L
merge_sort(L)
L

## Simulation

In [ ]:
def merge_sort_nb_of_steps(L):
    return _merge_sort_nb_of_steps(L, list(L), 0, len(L), 0) + 1

def _merge_sort_nb_of_steps(L1, L2, start, end, depth):
    if end == start:
        return 0
    if end == start + 1:
        return 1
    half_length = start + (end - start) // 2
    nb_of_steps =\
            _merge_sort_nb_of_steps(L2, L1, start, half_length, depth + 1)\
            + _merge_sort_nb_of_steps(L2, L1, half_length, end, depth + 1) + 1
    i = i1 = start
    i2 = half_length
    while i1 < half_length and i2 < end:
        nb_of_steps += 3
        if L2[i1] <= L2[i2]:
            L1[i] = L2[i1]
            i1 += 1
        else:
            L1[i] = L2[i2]
            i2 += 1
        i += 1
    for j in range(i1, half_length):
        L1[i] = L2[j]
        i += 1
    for j in range(i2, end):
        L1[i] = L2[j]
        i += 1
    return nb_of_steps + (half_length - i1 + end - i2) * 2

def trace_merge_sort(L):
    all_blacks = initialise_colouring(L, False)
    display_2(L, all_blacks, L, all_blacks)
    yield
    yield from _trace_merge_sort(L, list(L), 0, len(L), 0)
    
def _trace_merge_sort(L1, L2, start, end, depth):
    item_colours_1 = initialise_colouring(L1, False)
    item_colours_2 = deepcopy(item_colours_1)
    if end == start:
        return
    if end == start + 1:
        if depth == 0:
            item_colours_1[start]['color'] = 'green'
            display_2(L1, item_colours_1, L2, item_colours_2)
            yield
        else:
            item_colours_1[start]['color'] = item_colours_2[start]['color'] =\
                                                                         'blue'
            if depth % 2:
                display_2(L2, item_colours_2, L1, item_colours_1)
            else:
                display_2(L1, item_colours_1, L2, item_colours_2)
            yield
        return
    half_length = start + (end - start) // 2
    yield from _trace_merge_sort(L2, L1, start, half_length, depth + 1)
    yield from _trace_merge_sort(L2, L1, half_length, end, depth + 1)
    for i in range(start, end):
        item_colours_1[i]['color'] = item_colours_2[i]['color'] = 'blue'
    if depth % 2:
        display_2(L2, item_colours_2, L1, item_colours_1)
    else:
        display_2(L1, item_colours_1, L2, item_colours_2)        
    yield
    i = start
    i1 = start
    i2 = half_length
    while i1 < half_length and i2 < end:
        item_colours_2[i1]['color'] = item_colours_2[i2]['color'] = 'yellow'
        item_colours_2[i1]['attrs'] = item_colours_2[i2]['attrs'] = 'bold'
        if depth % 2:
            display_2(L2, item_colours_2, L1, item_colours_1)
        else:
            display_2(L1, item_colours_1, L2, item_colours_2)
        yield
        item_colours_2[i1]['attrs'] = item_colours_2[i2]['attrs'] = ''
        select_i1 = L2[i1] <= L2[i2]
        j1, j2 = (i1, i2) if select_i1 else (i2, i1)
        item_colours_2[j1]['color'] = 'red'
        item_colours_2[j2]['color'] = 'blue'
        L1[i] = '*'
        if depth % 2:
            display_2(L2, item_colours_2, L1, item_colours_1)
        else:
            display_2(L1, item_colours_1, L2, item_colours_2)
        yield
        L1[i] = L2[j1]
        item_colours_2[j1]['color'] = 'blue'
        if depth % 2:
            item_colours_1[i]['color'] = 'yellow'
            display_2(L2, item_colours_2, L1, item_colours_1)
            yield
            item_colours_1[i]['color'] = 'blue'
        else:
            item_colours_1[i]['color'] = depth and 'yellow' or 'green'
            display_2(L1, item_colours_1, L2, item_colours_2)
            yield
            if depth:
                item_colours_1[i]['color'] = 'blue'
        if select_i1:
            i1 += 1
        else:
            i2 += 1
        i += 1
    yield from transfer_end_of_list(i, i1, half_length, L1, item_colours_1, L2,
                                    item_colours_2, depth
                                   )
    yield from transfer_end_of_list(i + half_length - i1, i2, end, L1,
                                    item_colours_1, L2, item_colours_2, depth
                                   )

def transfer_end_of_list(i, j, bound, L1, item_colours_1, L2, item_colours_2,
                         depth
                        ):
    while j < bound:
        item_colours_2[j]['color'] = 'red'
        L1[i] = '*'
        if depth % 2:
            display_2(L2, item_colours_2, L1, item_colours_1)
        else:
            display_2(L1, item_colours_1, L2, item_colours_2)
        yield
        L1[i] = L2[j]
        item_colours_2[j]['color'] = 'blue'
        if depth % 2:    
            item_colours_1[i]['color'] = 'yellow'
            display_2(L2, item_colours_2, L1, item_colours_1)
            yield
            item_colours_1[i]['color'] = 'blue'
        else:
            item_colours_1[i]['color'] = depth and 'yellow' or 'green'
            display_2(L1, item_colours_1, L2, item_colours_2)
            yield
            if depth:
                item_colours_1[i]['color'] = 'blue'
        j += 1
        i += 1

In [ ]:
widgets.VBox([TraceSorting(trace_merge_sort, merge_sort_nb_of_steps).tabs])

## Number of comparisons in the worst case

The number of comparisons in the worst case is $N\llcorner\log_2(N)\lrcorner-2^{\llcorner\log_2(N)\lrcorner}+1$. To help verify the claim, take as example $N=9$ and $L=[8,4,6,2,7,3,5,9,1]$. The recursive calls and what segments of $L$ are changed to when the calls return can be represented by the binary tree below. This is a worst case because whenever two sorted lists $L_1$ and $L_2$ are merged, the resulting list consists of $L_2$'s first element, followed by $L_1$'s first element, followed by $L_2$'s second element if it exists, followed by $L_1$'s second element if it exists, followed by $L_2$'s third element if it exists... so either $L_2$ has an odd number of elements the last of which is the only one that remains after $L_1$'s last element has been processed, or $L_1$ has an even number of elements in which case $L_1$'s last element is the only one that remains after $L_2$'s last element has been processed.

<div><img src="Illustrations/merge_sort_2.pdf" width="400"/></div>

The tree has a height of 4, and down to a depth of 3, it is complete, with $2^0+2^1+2^2+2^3=2^4-1=15$ nodes.

In full generality, let $T$ denote the tree of recursive calls that are performed by Merge sort on a list $L$, labelling each node with the size of the associated sublist, and let $h$ denote $T$'s height; so with the previous example, $T$ is as follows.

<div><img src="Illustrations/merge_sort_3.pdf" width="400"/></div>

First, let us check that for all $h'<h$, the labels of the nodes on level $h'$ of the tree differ by at most 1. That is trivial for level 0. Proof for the other levels is by induction, so let $h'\in\{1,\dots,h-1\}$ be given, and assume that the property holds on level $h'-1$. Let $a$ and $b$ be the labels of two nodes on level $h'$. The labels of their parents are amongst $\llcorner\frac{a}{2}\lrcorner$, $\ulcorner\frac{a}{2}\urcorner$, $\llcorner\frac{b}{2}\lrcorner$ and $\ulcorner\frac{b}{2}\urcorner$. Without loss of generality, suppose that $a\leq b$. It suffices to verify that $\ulcorner\frac{b}{2}\urcorner-\llcorner\frac{a}{2}\lrcorner\leq 1$. By inductive hypothesis, $b\leq a+1$, hence

\begin{equation*}
\ulcorner\frac{b}{2}\urcorner-\llcorner\frac{a}{2}\lrcorner\leq\ulcorner\frac{a+1}{2}\urcorner-\llcorner\frac{a}{2}\lrcorner=\llcorner\frac{a+2}{2}\lrcorner-\llcorner\frac{a}{2}\lrcorner=\llcorner\frac{a}{2}\lrcorner+1-\llcorner\frac{a}{2}\lrcorner=1
\end{equation*}

and we are done.

Now take a node on level $h$. Being a leaf, it is labelled with 1. Its parent, on level $h-1$, is labelled with with 2 or more, and its grand-parent, on level $h-2$, is labelled with 3 or more, which together with what has been established,  implies that all nodes on level $h-2$ are labelled with 2 or more. Hence $T$ is complete all the way down to level $h-1$ (at least). We infer that $h=\llcorner\log_2(N)\lrcorner$.

Assume that $L$ is a list for which Merge sort performs worst, for instance because it is of the same kind as the one given as example, so any two lists to merge are entwined; more precisely, merging a sublist of size $n_1$ with a sublist of size $n_2$ always requires $n_1+n_2-1$ comparisons. Let $h'<h$ be given, and let $a_1$, $a_2$, ..., $a_{2^{h'}}$ be the values of the nodes on level $h'$. The total number of comparisons to obtain the lists associated with these nodes is therefore $\Sigma_{1\leq i\leq 2^{h'}}(a_i-1)$. Obviously, $\Sigma_{1\leq i\leq 2^{h'}}a_i=N$. Hence the total number of comparisons performed by Merge sort on $L$ is

\begin{equation*}
\Sigma_{i=0}^{h-1}(N-2^i)=Nh-2^h+1=N\llcorner\log_2(N)\lrcorner-2^{\llcorner\log_2(N)\lrcorner}+1
\end{equation*}

as claimed.

# Quick sort

## Description

If $L$ has less than 2 elements then $L$ is sorted. Otherwise, let $e$ denote $L$'s first element, the _pivot_, and let $L'$ denote $L$ without the pivot. Quick sort possibly swaps pairs of elements in $L'$ in such a way that $L$ becomes a list of the form $L_1+L_2$ such that:

* $L_1$ consists of $e$ followed by elements all of which are smaller than or equal to the pivot;
* $L_2$ consists of elements all of which are greater than or equal to the pivot.

If $L_1$ is $[e]$ then the pivot is found out to be in place and $L_2$ is sorted recursively. Otherwise, the pivot is swapped with $L_1$'s last element, which puts it in place, and it remains to sort recursively $L_1$ as modified after the swap and excluding the pivot, and to sort recursively $L_2$. In any case, this is achieved as follows. The list $L'$ is scanned from left to right until an element strictly greater than the pivot is found, if any.

1. If no such element is found then $L_1$ is $L$ and $L_2$ is empty.
2. If such an element is found, say $e_1$, then $L'$ is scanned from right to left until an element strictly smaller than the pivot is found, if any.
    1. If no such element is found then $L_1$ is the initial segment of $L$ that ends before $e_1$, so it is $[e]$ if $e_1$ comes right after the pivot, and $L_2$ is the final segment of $L$ that starts with $e_1$.
    2. If such an element is found, say $e_2$, then it is swapped with $e_1$.
        1. If these two elements are next to each other then $L_1$ is the initial segment of $L$ that ends with $e_2$ (just swapped with $e_1$) and $L_2$ is the final segment of $L$ that starts with $e_1$ (just swapped with $e_2$).
        2. Otherwise, the procedure that has been applied to $L'$ is applied to the sublist of $L'$ consisting of the elements that sit between $e_2$ (just swapped with $e_1$) excluded and $e_1$ (just swapped with $e_2$) excluded, to be possibly modified and in any case be of the form $L_1'+L_2'$ so that
            1. $L_1$ is the initial segment of $L$ that ends with $e_2$ (previously swapped with $e_1$) concatenated with $L'_1$;
            2. $L_2$ is $L'_2$ concatenated with the final segment of $L$ that starts with $e_1$ (previously swapped with $e_2$).

We illustrate the procedure as a sequence of 3 diagrams displayed horizontally.

* The first diagram highlights the pivot in light red and shows the pairs of swaps that have to be performed in $L'$, if any.
* The second diagram shows $L_1$ and $L_2$, in light magenta and cyan, respectively, and the swap that has to be performed between the pivot and $L_1$'s last element if $L_1$ is not empty.
* The third diagram shows the pivot in green, correctly positioned between the two sublists that will be sorted recursively.

Here is an illustration of Case 1:

<div><img src="Illustrations/quick_sort_1.pdf" width="600"/></div>

Here are illustrations of Case 2A:

<div><img src="Illustrations/quick_sort_2A.pdf" width="600"/></div>

Here are illustrations of Case 2Ba:

<div><img src="Illustrations/quick_sort_2Ba.pdf" width="600"/></div>

Here are illustrations of Case 2Bb:

<div><img src="Illustrations/quick_sort_2Bb.pdf" width="800"/></div>




## Implementation

In [ ]:
def quick_sort(L):
    quicksort(L, 0, len(L) - 1)

def quicksort(L, start, last):
    if last - start < 1:
        return
    split_point = partition(L, start, last)
    quicksort(L, start, split_point - 1)
    quicksort(L, split_point + 1, last)

def partition(L, start, end):
    pivot_value = L[start]
    left_mark = start + 1
    right_mark = end
    while True:
        while left_mark <= right_mark and L[left_mark] <= pivot_value:
            left_mark += 1
        while right_mark > left_mark and L[right_mark] >= pivot_value:
            right_mark -= 1
        # All values till position left_mark excluded are
        # at most equal to the pivot.
        # All values from position right_mark excluded are
        # at least equal to the pivot.
        # The value at position right_mark is greater than the pivot.
        if left_mark == right_mark:
            right_mark -= 1
        if right_mark == start:
            return start
        # Either the penultimate if statement was true
        # and its body has been executed, or two values
        # next to each other have just been swapped.
        if left_mark == right_mark + 1:
            L[start], L[right_mark] = L[right_mark], L[start]
            return right_mark
        L[left_mark], L[right_mark] = L[right_mark], L[left_mark]

In [ ]:
L = list(range(1, 10))
shuffle(L)
L
quick_sort(L)
L

## Simulation

In [ ]:
def quick_sort_nb_of_steps(L):
    return _quick_sort_nb_of_steps(L, 0, len(L) - 1) + 1

def _quick_sort_nb_of_steps(L, start, last):
    if last < start:
        return 0
    if last == start:
        return 1
    nb_of_steps, split_point = partition_nb_of_steps(L, start, last)
    if split_point != start:
        nb_of_steps += 1
    nb_of_steps += _quick_sort_nb_of_steps(L, start, split_point - 1)
    if start < split_point < last:
        nb_of_steps += 1
    return nb_of_steps + _quick_sort_nb_of_steps(L, split_point + 1, last)

def partition_nb_of_steps(L, start, end):
    nb_of_steps = 0
    pivot_value = L[start]
    left_mark = start + 1
    right_mark = end
    while True:
        while left_mark <= right_mark and L[left_mark] <= pivot_value:
            nb_of_steps += 1
            left_mark += 1
        if left_mark <= right_mark:
            nb_of_steps += 1
        while right_mark > left_mark and L[right_mark] >= pivot_value:
            nb_of_steps += 1
            right_mark -= 1
        if left_mark == right_mark:
            right_mark -= 1
        if right_mark == start:
            return nb_of_steps + 1, start
        if left_mark == right_mark + 1:
            L[start], L[right_mark] = L[right_mark], L[start]
            return nb_of_steps + 2, right_mark
        nb_of_steps += 3
        L[left_mark], L[right_mark] = L[right_mark], L[left_mark]
        left_mark += 1
        right_mark -= 1

def trace_quick_sort(L):
    item_colours = initialise_colouring(L)
    display(L, item_colours)
    yield
    yield from _trace_quick_sort(L, item_colours, 0, len(L) - 1)

def _trace_quick_sort(L, item_colours, start, last):
    if last < start:
        return
    if last == start:
        item_colours[start]['color'] = 'green'
        display(L, item_colours)
        yield
        return
    for i in range(start, last + 1):
        item_colours[i]['color'] = 'blue'
    yield from trace_partition(L, item_colours, start, last)
    split_point = trace_partition.split_point
    for i in range(split_point + 1, last + 1):
         item_colours[i].pop('color')
    if split_point != start:
        for i in range(start, split_point):
            item_colours[i]['color'] = 'blue'
        display(L, item_colours)
        yield
    yield from _trace_quick_sort(L, item_colours, start, split_point - 1)
    if start < split_point < last:
        for i in range(split_point + 1, last + 1):
            item_colours[i]['color'] = 'blue'
        display(L, item_colours)
        yield
    yield from _trace_quick_sort(L, item_colours, split_point + 1, last)

def trace_partition(L, item_colours, start, end):
    pivot_value = L[start]
    left_mark = start + 1
    right_mark = end
    while True:
        item_colours[start]['color'] = 'yellow'
        item_colours[start]['attrs'] = 'bold'
        while left_mark <= right_mark and L[left_mark] <= pivot_value:
            item_colours[left_mark]['color'] = 'yellow'
            item_colours[left_mark]['attrs'] = 'bold'
            display(L, item_colours)
            yield
            item_colours[left_mark]['color'] = 'blue'
            item_colours[left_mark]['attrs'] = ''
            left_mark += 1
        if left_mark <= right_mark:
            item_colours[left_mark]['color'] = 'yellow'
            item_colours[left_mark]['attrs'] = 'bold'
            display(L, item_colours)
            yield
            item_colours[left_mark]['color'] = 'blue'
            item_colours[left_mark]['attrs'] = ''
        while right_mark > left_mark and L[right_mark] >= pivot_value:
            item_colours[right_mark]['color'] = 'yellow'
            item_colours[right_mark]['attrs'] = 'bold'
            display(L, item_colours)
            yield
            item_colours[right_mark]['color'] = 'blue'
            item_colours[right_mark]['attrs'] = ''
            right_mark -= 1
        if left_mark == right_mark:
            right_mark -= 1
        if right_mark < left_mark:
            trace_partition.split_point = right_mark
            item_colours[start]['attrs'] = ''
        if right_mark == start:
            item_colours[start]['color'] = 'green'
            display(L, item_colours)
            yield
            return
        if left_mark == right_mark + 1:
            item_colours[start]['color'] =\
                    item_colours[right_mark]['color'] = 'red'
            display(L, item_colours)
            yield
            L[start], L[right_mark] = L[right_mark], L[start]
            item_colours[start]['color'] = 'yellow'
            item_colours[right_mark]['color'] = 'green'
            display(L, item_colours)
            yield
            return
        item_colours[right_mark]['color'] = 'yellow'
        item_colours[right_mark]['attrs'] = 'bold'
        display(L, item_colours)
        yield
        item_colours[start]['color'] = 'blue'
        item_colours[start]['attrs'] = item_colours[right_mark]['attrs'] = ''
        item_colours[left_mark]['color'] =\
                        item_colours[right_mark]['color'] = 'red'
        display(L, item_colours)
        yield
        L[left_mark], L[right_mark] = L[right_mark], L[left_mark]
        item_colours[left_mark]['color'] =\
                    item_colours[right_mark]['color'] = 'yellow'
        display(L, item_colours)
        yield
        item_colours[left_mark]['color'] =\
                    item_colours[right_mark]['color'] = 'blue'
        left_mark += 1
        right_mark -= 1

In [ ]:
widgets.VBox([TraceSorting(trace_quick_sort, quick_sort_nb_of_steps).tabs])

## Number of comparisons in the worst case

If $L$ is sorted then $L$'s first element will be found out to be in place after $N-1$ comparisons, then $L$'s last $N-1$ elements will be sorted recursively and $L$'s second element will be found out to be in place after $N-2$ comparisons..., and eventually $L$'s last 2 elements will be sorted recursively and $L$'s penultimate element will be found out to be in place after 1 comparison, for a total of $\frac{N(N-1)}{2}$ comparisons.

## Average number of comparisons

Let us verify that on average, Quick sort performs $2(N+1)\Bigl(\Sigma_{i=2}^{N+1}\frac{1}{i}+\frac{1}{N+1}-1\Bigr)$ comparisons. Since $\lim_{n\rightarrow\infty}\Sigma_{i=1}^n\frac{1}{i}=\ln(N)+\gamma$ with $\gamma$ denoting the _Euler-Mascheroni_ constant, that is $O\bigl(N\log(N)\bigr)$ comparisons on average.

Let $Q_n$ denote the number of comparisons performed by Quicksort on a list of size $n$, $n\in\mathbb N$. Suppose that $N>1$. The pivot has an equal chance of ending up at any of the $N$ positions. When it ends up at index $i$ in $L$, $0\leq i<N$, there are two lists of size $i$ and $N-i-1$ to sort recursively, that will require $Q_{i}$ and $Q_{N-i-1}$ comparisons on average, respectively. The pivot is put in place after $N-1$ comparisons have been performed, one between the pivot and each of $L'$'s members. Hence for all $n>1$ we have the equation:

\begin{equation}\label{eq1}\tag{1}
Q_n=n-1+\frac{1}{n}\Sigma_{i=0}^{n-1}(Q_i+Q_{n-i-1}).
\end{equation}

Since for all $i<n$, $Q_{n-i-1}=Q_i$, (\ref{eq1}) can be rewritten as

\begin{equation}\label{eq2}\tag{2}
Q_n=n-1+\frac{2}{n}\Sigma_{i=0}^{n-1}Q_i.
\end{equation}

Multiplying both sides of (\ref{eq2}) by $n$ yields

\begin{equation}\label{eq3}\tag{3}
nQ_n=n(n-1)+2\Sigma_{i=0}^{n-1}Q_i.
\end{equation}

When $n>1$, (\ref{eq3}) also yields

\begin{equation}\label{eq4}\tag{4}
(n-1)Q_{n-1}=(n-1)(n-2)+2\Sigma_{i=0}^{n-2}Q_i.
\end{equation}

Subtracting side by side (\ref{eq4}) from (\ref{eq3}), we obtain

\begin{equation*}
nQ_n-(n-1)Q_{n-1}=n(n-1)-(n-1)(n-2)+2Q_{n-1},
\end{equation*}

which can be simplified to

\begin{equation*}
nQ_n-(n-1)Q_{n-1}=2(n-1)+2Q_{n-1},
\end{equation*}

which can be further simplified to

\begin{equation*}
nQ_n=(n+1)Q_{n-1}+2(n-1),
\end{equation*}

from which we derive

\begin{equation}\label{eq5}\tag{5}
\frac{Q_n}{n+1}=\frac{Q_{n-1}}{n}+\frac{2(n-1)}{n(n+1)}.
\end{equation}

Using versions of (\ref{eq5}) where $n$ is replaced by $n-1$, $n-2$, ..., 2, together with the obvious fact that $Q_1=0$, and

* replacing in (\ref{eq5}) $\frac{Q_{n-1}}{n}$ by the right hand side of the version of (\ref{eq5}) where $n-1$ replaces $n$,
* then replacing in the obtained equation $\frac{Q_{n-2}}{n-1}$ by the right hand side of the version of (\ref{eq5}) where $n-2$ replaces $n$,
* ...

we eventually get

\begin{equation}\label{eq6}\tag{6}
\frac{Q_n}{n+1}=2\Sigma_{i=1}^n\frac{i-1}{i(i+1)}.
\end{equation}

Since for all $i>0$, $\frac{i-1}{i(i+1)}=\frac{2}{i+1}-\frac{1}{i}$, it follows from (\ref{eq6}) that

\begin{equation*}
\frac{Q_n}{n+1}=\Sigma_{i=1}^n\frac{4}{1+i}-\Sigma_{i=1}^n\frac{2}{i},
\end{equation*}

hence

\begin{equation*}
\frac{Q_n}{n+1}=\Sigma_{i=2}^n\frac{2}{i}+\frac{4}{n+1}-2.
\end{equation*}

Theorefore, as claimed,

\begin{equation*}
Q_N=2(N+1)\Bigl(\Sigma_{i=2}^{N+1}\frac{1}{i}+\frac{1}{N+1}-1\Bigr).
\end{equation*}

# Heap sort

## Description

To understand Heap sort, it is necessary to map lists to _complete binary trees_: read the values of the list from left to right and let them make up the values of the nodes of a binary tree, read from top to bottom and on a given level read from left to right, filling the tree so that all levels except possibly the last one are full, and having all nodes on the last level as much to the left as possible. For instance, take $[1, 10, 3, 7, 6, 2, 4, 9, 8, 5]$ as an example for $L$. It is mapped to the tree on the left below.

<div><img src="Illustrations/heap_sort_1.pdf" width="600"/></div>

The tree on the right has the same structure, that of all complete binary trees with 10 nodes, but it is moreover _heap ordered_, because the value of any inner node is at least equal to the value of its unique child or its two children: $10\geq\min(10,9)$, $9\geq\min(8,6)$, $4\geq\min(2,3)$, $8\geq\min(7,1)$, and $6\geq 5$. Heap sort will first _heapify_ the tree that the list to sort has been mapped to, that is, move the values around so that the tree becomes heap ordered. Heapifying the tree on the left above will precisely change it to the tree on the right above. Hence for our example, Heap sort will first change $L$ to $[10,9,4,8,6,2,3,7,1,5]$. This will be done using the technique of _bubbling down_ in a tree the value of a node of the tree. This technique will be used again in the second part of the algorithm. Let us first describe the technique of bubbling down the value at the root $R$ of a tree $T$ (for the value of any other node, the technique just operates on the subtree rooted at that node). Bubbling down in $T$ the value $x$ at $R$ is performed recursively as follows.

* If $T$ has no other node but $R$, or if $x$ is greater than or equal to the value of any child of $R$, then nothing is done.
* Otherwise, there are two cases.
    * $R$ has a unique child $R'$.
    * $R$ has two children, a left child $R_1$ and a right child $R_2$. Let $R$ be the child with the largest value, or in case both values are equal, arbitrarily let $R'$ be $R_1$.

In any case, let $R$ and $R'$ swap their values, so $x$ becomes the value of the root of $R'$. Bubble $x$ down in the tree rooted at $R'$.

It is immediately verified by induction that whether $R$ has a unique child that is the root of a heap ordered binary tree, or $R$ has two children each of which is the root of a heap ordered binary tree, bubbling $x$ down in $T$ makes $T$ itself heap ordered. It follows that a tree can be made heap ordered by bubbling down the values of all of its node, proceeding from the rightmost lowest node to the root (processing the nodes in reverse order of the standard enumeration previously described). The first nodes to process are the leaves, for which there is obviously nothing to do, so the bubbling down can actually start at the rightmost, leftmost inner node, which for our example of the tree $T$ that the list $[10,9,4,8,6,2,3,7,1,5]$ is mapped to, is the node with the value 6. This node has only one child, whose value is 5, so the subtree rooted at that node is heap ordered. The next node to process is the node whose value is 7, that is bubbled down as illustrated below, changing $T$ to the tree depicted on the right.

<div><img src="Illustrations/heap_sort_2.pdf" width="600"/></div>

Next comes the node whose value is 3:

<div><img src="Illustrations/heap_sort_3.pdf" width="600"/></div>

The subtree rooted at the node whose value is 10 is heap-ordered already, so to be done with the first stage of the algorithm, it remains to bubble 1 down. Observe that bubbling down an element determines a path, here (1, 10, 9, 8), that starts with the node under consideration and that goes down the tree, always chosing the largest value when having to go down either left or right, and stopping when reaching either a leaf or a node that has no child with a value that is stricly larger than the value being bubbled down. The effect is to bring the value being bubbled down to the end of the path, and to lift all other values along the path one level up.

<div><img src="Illustrations/heap_sort_4.pdf" width="600"/></div>

At this stage, $T$ has been heapified and $L$ has been correspondingly changed to $[10,9,4,8,6,2,3,7,1,5]$. Observe that $L$'s largest value, namely 10, is now the list's first element, or equivalently, is at the root of the tree. Having to become the list's last element, it can be put in place by swapping the list's first and last elements. Using the tree representation, the effect of the swap is shown in red for 5 and green for 10, which is now in place, in the tree on the left below. At this point, we have to consider that 10 is no longer part of the (active part of the) list and the (active part of the) tree any more, do as if  the list had been truncated of its last element, do as if the tree had been truncated of its last node. The 9-node tree is no longer heap ordered, but its proper subtrees are, so to heapify the tree it suffices to bubble 5 down, as determined by the path represented by the nodes in bright and light red. This results in the tree on the right below.

<div><img src="Illustrations/heap_sort_5.pdf" width="600"/></div>

The procedure is repeated to swap the value at the root of the tree, namely 9, then 8, then 7, with the value at the last node of the (active part of the) tree whose size changes from 9 to 8, then 7, then 6 as it "loses" the nodes with the values that have been put in place:

<div><img src="Illustrations/heap_sort_6.pdf" width="600"/></div>

Repeating the procedure 3 more time gets us closer:

<div><img src="Illustrations/heap_sort_7.pdf" width="600"/></div>

At this point 3 can be put in place and the (active part of the) tree is reduced to two nodes, with 1 at the root and 2 at the left child of the root. Bubbling 1 down leaves 1 and 2 in place, and we are done:

<div><img src="Illustrations/heap_sort_8.pdf" width="600"/></div>

To implement Heap sort, it is not necessary to use trees as a data structure and methods on trees. Operating directly on the list to sort is easy: it suffices to observe that if a value $v$ is at index $i$ in the list $L$ to sort, $T$ is the heap ordered complete binary tree that $L$ is mapped to, and $v$ is at node $V$ in $T$, then:

* either $V$ is a leaf, in which case $i$ is at least equal to $L$'s length,
* or $V$ has a left child $V'$ and no right child, in which case the value at $V'$ is at index $2i+1$ in $L$ and $2i+2$ is equal to $L$'s length,
* or $V$ has both a left child $V_1$ and a right child $V_2$, in which case the value at $V_1$ is at index $2i+1$ in $L$ and the value at $V_2$ is at index $2i+2$ in $L$.

## Implementation

In [ ]:
def heap_sort(L):
    if len(L) < 2:
        return
    for i in range(len(L) // 2 - 1, -1, -1):
        bubble_down(L, i, len(L))
    for i in range(len(L) - 1, 2, -1):
        L[0], L[i] = L[i], L[0]
        bubble_down(L, 0, i)
    if len(L) > 2:
        L[0], L[2] = L[2], L[0]
    if L[0] > L[1]:
        L[0], L[1] = L[1], L[0]

def bubble_down(L, i, length):
    while True:
        index_of_greatest_child = 2 * i + 1
        if index_of_greatest_child >= length:
            return
        if (index_of_greatest_child < length - 1 and
                 L[index_of_greatest_child] < L[index_of_greatest_child + 1]):
            index_of_greatest_child = index_of_greatest_child + 1
        if L[i] >= L[index_of_greatest_child]:
            return
        else:
            L[i], L[index_of_greatest_child] = L[index_of_greatest_child], L[i]
        i = index_of_greatest_child

In [ ]:
L = list(range(1, 10))
shuffle(L)
L
heap_sort(L)
L

## Simulation

In [ ]:
def heap_sort_nb_of_steps(L):
    nb_of_steps = 1
    for i in range(len(L) // 2 - 1, -1, -1):
        nb_of_steps += bubble_down_nb_of_steps(L, i, len(L))
    for i in range(len(L) - 1, 2, -1):
        nb_of_steps += 2
        L[0], L[i] = L[i], L[0]
        if i > 2:
            nb_of_steps += bubble_down_nb_of_steps(L, 0, i)
    if len(L) > 2:
        nb_of_steps += 2
        L[0], L[2] = L[2], L[0]
    nb_of_steps += 2
    if L[0] > L[1]:
        L[0], L[1] = L[1], L[0]
    return nb_of_steps

def bubble_down_nb_of_steps(L, i, length):
    nb_of_steps = 0
    while True:
        index_of_greatest_child = 2 * i + 1
        if index_of_greatest_child >= length:
            break
        if index_of_greatest_child < length - 1:
            nb_of_steps += 1
            if L[index_of_greatest_child] < L[index_of_greatest_child + 1]:
                index_of_greatest_child = index_of_greatest_child + 1
        if L[i] >= L[index_of_greatest_child]:
            nb_of_steps += 1
            break
        else:
            nb_of_steps += 2
            L[i], L[index_of_greatest_child] = L[index_of_greatest_child], L[i]
        i = index_of_greatest_child
    return nb_of_steps

def trace_heap_sort(L):
    item_colours = initialise_colouring(L)
    display(L, item_colours)
    yield
    for i in range(len(L) // 2 - 1, -1, -1):
        yield from trace_bubble_down(L, item_colours, i, len(L))
    for i in range(len(L) - 1, 2, -1):
        item_colours[0]['color'] = item_colours[i]['color'] = 'red'
        item_colours[0]['attrs'] = item_colours[i]['attrs'] = 'bold'
        display(L, item_colours)
        yield
        L[0], L[i] = L[i], L[0]
        item_colours[0]['attrs'] = item_colours[i]['attrs'] = ''
        item_colours[0]['color'] = 'yellow'
        item_colours[i]['color'] = 'green'
        display(L, item_colours)
        yield
        item_colours[0]['color'] = 'blue'
        if i > 2:
            yield from trace_bubble_down(L, item_colours, 0, i)
    if len(L) > 2:
        item_colours[0]['color'] = item_colours[2]['color'] = 'red'
        item_colours[0]['attrs'] = item_colours[2]['attrs'] = 'bold'
        display(L, item_colours)
        yield
        L[0], L[2] = L[2], L[0]
        item_colours[0]['attrs'] = item_colours[2]['attrs'] = ''
        item_colours[0]['color'] = 'yellow'
        item_colours[2]['color'] = 'green'
        display(L, item_colours)
        yield
    item_colours[0]['attrs'] = item_colours[1]['attrs'] = 'bold'
    if L[0] > L[1]:
        item_colours[0]['color'] = item_colours[1]['color'] = 'red'
        display(L, item_colours)
        yield
        L[0], L[1] = L[1], L[0]
    else:
        item_colours[0]['color'] = item_colours[1]['color'] = 'yellow'
        display(L, item_colours)
        yield
    item_colours[0]['attrs'] = item_colours[1]['attrs'] = ''
    item_colours[0]['color'] = item_colours[1]['color'] = 'green'
    display(L, item_colours)
    yield

def trace_bubble_down(L, item_colours, i, length):
    while True:
        index_of_greatest_child = 2 * i + 1
        if index_of_greatest_child >= length:
            break
        if index_of_greatest_child < length - 1:
            item_colours[index_of_greatest_child]['color'] =\
              item_colours[index_of_greatest_child + 1]['color'] = 'yellow'
            item_colours[index_of_greatest_child]['attrs'] =\
              item_colours[index_of_greatest_child + 1]['attrs'] = 'bold'
            display(L, item_colours)
            yield
            if L[index_of_greatest_child] < L[index_of_greatest_child + 1]:
                item_colours[index_of_greatest_child]['color'] = 'blue'
                item_colours[index_of_greatest_child]['attrs'] = ''
                index_of_greatest_child = index_of_greatest_child + 1
            else:
                item_colours[index_of_greatest_child + 1]['color'] = 'blue'
                item_colours[index_of_greatest_child + 1]['attrs'] = ''
        if L[i] >= L[index_of_greatest_child]:
            item_colours[i]['color'] =\
                    item_colours[index_of_greatest_child]['color'] = 'yellow'
            item_colours[i]['attrs'] =\
                    item_colours[index_of_greatest_child]['attrs'] = 'bold'
            display(L, item_colours)
            yield
            item_colours[i]['color'] =\
                    item_colours[index_of_greatest_child]['color'] = 'blue'
            item_colours[i]['attrs'] =\
                    item_colours[index_of_greatest_child]['attrs'] = ''
            break
        else:
            item_colours[i]['color'] =\
                    item_colours[index_of_greatest_child]['color'] = 'red'
            item_colours[i]['attrs'] =\
                    item_colours[index_of_greatest_child]['attrs'] = 'bold'
            display(L, item_colours)
            yield
            L[i], L[index_of_greatest_child] = L[index_of_greatest_child], L[i]
            item_colours[i]['color'] =\
                    item_colours[index_of_greatest_child]['color'] = 'yellow'
            item_colours[i]['attrs'] =\
                    item_colours[index_of_greatest_child]['attrs'] = ''
            display(L, item_colours)
            yield
            item_colours[i]['color'] =\
                    item_colours[index_of_greatest_child]['color'] = 'blue'
        i = index_of_greatest_child

In [ ]:
widgets.VBox([TraceSorting(trace_heap_sort, heap_sort_nb_of_steps).tabs])

## Complexity of the number of comparisons in the worst case

Let $T$ be the heap ordered complete binary tree that the list $L$ to sort is mapped to. Since $T$ is complete, its height is $\llcorner\log_2(N)\lrcorner$. Bubbling $T$'s root down requires at most $2(\llcorner\log_2(N)\lrcorner-1)$ comparisons since for a node $V$ with two children, it is necessary to find out which of the two children holds the larger value and then compare that value to the value at $V$ to possibly bubble the latter one level down. To heapify $T$, $\llcorner\frac{N}{2}\lrcorner$ elements need to be bubbled down. Then $T$'s root needs to be bubbled down $N-1$ times. Hence the number of comparisons in the worst case is of complexity $O\bigl(N\log(N)\bigr)$.

# Batcher sort

## Description

A _nonadaptive_ sorting algorithm is equivalent to a sequence of instructions of the form:

    order(L, i_0, j_0)
    order(L, i_1, j_1)
    ...
    order(L, i_k, j_k)

for some natural number `k` and natural numbers `i_0`, `i_1`, `j_1`, ..., `i_k`, `j_k` smaller than $N$, where `order()` is defined as:

    def order(L, i, j):
        if L[i] > L[j]:
            L[i], L[j] = L[j], L[i]

So a nonadaptive sorting algorithm has the property that for a given length, the same sequence of comparisons is performed to sort any list of that length, each comparison of two values resulting or not in a swap of those values. For instance, for lists of length 4, such a sequence of instructions could be:

    order(L[0], L[1])
    order(L[2], L[3])
    order(L[0], L[2])
    order(L[1], L[3])
    order(L[1], L[2])

This sequence of instructions expresses that

* the smallest element is the least of the smallest of the first two and the smallest of the last two,
* the largest element is the greatest of the largest of the first two and the largest of the last two, and
* the middle elements might have to be swapped.

It can be represented by the _sorting network_ that follows.

<div><img src="Illustrations/batcher_sort_1.pdf" width="250"/></div>

The first two comparisons could be performed in parallel, and then the next two comparisons could also be performed in parallel.

Batcher sort is a nonadaptive algorithm designed for lists whose length is a power of 2, so we now assume that $N$ is a power of 2. Recall that Merge sort splits an array in two halves, sorts both halves recursively, yielding two sorted halves $H_1$ and $H_2$, and then merges $H_1$ and $H_2$. When $N>2$, merging can be done by first _unshuffling_, or in Python terms unzipping, the data, for $N=8$ transforming

\begin{equation*}
\overbrace{x_1\ \ x_2\ \ x_3\ \ x_4}^{H_1}\ \ \overbrace{y_1\ \ y_2\ \ y_3\ \ y_4}^{H_2}
\end{equation*}
    
where $x_1\leq x_2\leq x_3\leq x_4$ and $y_1\leq y_2\leq y_3\leq y_4$ into

\begin{equation*}
\overbrace{x_1\ x_3\ y_1\ y_3}^{C_1}\ \ \ \overbrace{x_2\ x_4\ y_2\ y_4}^{C_2}
\end{equation*}

then ordering both new halves $C_1$ and $C_2$ separately

\begin{equation*}
\overbrace{u_1\ u_2\ u_3\ u_4}^{D_1}\ \ \ \overbrace{v_1\ v_2\ v_3\ v_4}^{D_2}
\end{equation*}

then _shuffling_, or in Python terms zipping, the resulting halves $D_1$ and $D_2$

\begin{equation*}
u_1\ \ v_1\ \ u_2\ \ v_2\ \ u_3\ \ v_3\ \ u_4\ \ v_4
\end{equation*}

and finally ordering the second and third elements, the fourth and fifth elements...

\begin{equation*}
u_1\ \ v_1\leftrightarrow u_2\ \ v_2\leftrightarrow u_3\ \ v_3\leftrightarrow u_4\ \ v_4
\end{equation*}

Here is why.

* After unshuffling, $C_1$ contains the smallest element from $H_1$, namely $x_1$, and the smallest element from $H_2$, namely $y_1$, with the smallest of the two, namely $u_1$, being put to first position after shuffling.
* After unshuffling, $C_2$ contains the largest element from $H_1$, namely $x_4$, and the largest element from $H_2$, namely $y_4$, with the largest of the two, namely $v_4$, being put to last position after shuffling.
* Assume that $N>2$, and suppose that the first $2i+1$ data, $0\leq i<\frac{N}{2}-1$, have been correctly put in place. What remains to put in place is in the sorted sequence $u_{2i+2}$, ..., $u_{\frac{N}{2}}$ from $D_1$ and in the sorted sequence $v_{2i+1}$, ..., $v_{\frac{N}{2}-1}$ from $D_2$. The last step of the procedure that has been described compares $u_{2i+2}$ and $v_{2i+1}$ and puts the smallest of the two in place. Without loss of generality, suppose it is $u_{2i+2}$  and assume for a contradiction that $v_{2i+1}$ is not the next smallest value, so $i<\frac{N}{2}-2$ and $u_{2i+3}$ is the next smallest value. If $u_{2i+2}$ and $u_{2i+3}$ both come from $H_1$ or both come from $H_2$, then there is some value in-between which belongs to $C_2$, hence has to be one of $v_{2i+1}$, ..., $v_{\frac{N}{2}-1}$, which is impossible. Hence $u_{2i+2}$ has to be the penultimate element from $H_1$, namely $x_{\frac{N}{2}-1}$, and $u_{2i+3}$ has to be the first element from $H_2$, namely $y_1$, or the other way around. But then $\frac{N}{2}-2$, an even number, of values have been put in place, in contradiction with the initial assumption. Hence the first $2i+3$ values have been correctly put in place.

Note that $C_1$ and $C_2$ themselves consist of two sorted halves, namely $[x_1, x_3]$ and $[y_1, y_3]$, respectively, hence the technique just described can be applied to the halves of $C_1$ and the halves of $C_2$. More generally, since $N$ is of the form $2^p$, a call to `order()` can be applied to $p$ consecutive sublists of $L$ of size 2, each such call being represented by the sorting network that follows

<div><img src="Illustrations/batcher_sort_2.pdf" width="90"/></div>

before the technique that has been described is applied to $\frac{p}{2}$ consecutive sublists of $L$ of size 2 if $p\geq 2$, then to $\frac{p}{4}$ consecutive sublists of $L$ of size 4 if $p\geq 3$... The sorting network $S$ for ach sublist of $L$ of size $2^i$, $0<i\leq p$, can be obtained by "copying and "entwining" the pattern of the sorting network for the previous size to both the $2^{i-1}$ even lines of $S$ and the $2^{i-1}$ odd lines of $S$, and then ordering the elements on lines 1 and 2, and if $N>4$ the elements on lines 3 and 4 and the elements on lines 5 and 6, and if $N>8$ the elements on lines 7 and 8... For lists of size 4, the sorting network is as follows.

<div><img src="Illustrations/batcher_sort_3.pdf" width="110"/></div>

For lists of size 8, the sorting network is as follows.

<div><img src="Illustrations/batcher_sort_4.pdf" width="180"/></div>

A pattern emerges. Putting everything together, and taking $N=32$ as an example, the sorting network is represented below and consists of:

* a stack of 16 networks of size 2 to sort 16 consecutive sublists of $L$ of size 2 each of whose 2 consecutive sublists of size 1 are trivially sorted,
* followed by a stack of 8 networks of size 4 to sort 8 consecutive sublists of $L$ of size 4 each of whose 2 consecutive sublists of size 2 are now sorted,
* followed by a stack of 4 networks of size 8 to sort 4 consecutive sublists of $L$ of size 8 each of whose 2 consecutive sublists of size 4 are now sorted,
* followed by a stack of 2 networks of size 16 to sort 2 consecutive sublists of $L$ of size 16 each of whose 2 consecutive sublists of size 8 are now sorted,
* followed by 1 network of size 32 to sort $L$ whose 2 consecutive sublists of size 16 are now sorted.

<div><img src="Illustrations/batcher_sort_5.pdf" width="800"/></div>

We enrich the previous diagram so it refers to the proposed implementation and helps understand it.

* `h` in the diagram is for `half_size` in the program.
* `s` in the diagram is for `span` in the program.
* `t` in the diagram is for `top` in the program.
* `group` in the program is the index of a "group of lines" some of which are skipped: the program skips every `skip`th group, as represented by the columns of numbers at the top of the diagram.

<div><img src="Illustrations/batcher_sort_6.pdf" width="800"/></div>

## Implementation

In [ ]:
# Assumes that len(L) is a power of 2
def batcher_sort(L):
    half_size, size = 1, 2
    while half_size < len(L):
        for group in range(0, len(L) // size):
            top = group * size
            for i in range(half_size):
                if L[top + i] > L[top + i + half_size]:
                    L[top + i], L[top + i + half_size] =\
                            L[top + i + half_size], L[top + i]
        span, double_span = half_size // 2, half_size
        while span:
            skip = half_size // span
            for group in range(len(L) // double_span):
                if (group + 1) % skip == 0:
                    continue
                top = span + group * double_span;
                for i in range(span):
                    if L[top + i] > L[top + i + span]:
                        L[top + i], L[top + i + span] =\
                                L[top + i + span], L[top + i]
            span //= 2
            double_span //= 2
        half_size *= 2
        size *= 2

In [ ]:
L = list(range(1, 17))
shuffle(L)
L
batcher_sort(L)
L

## Simulation

In [ ]:
def batcher_sort_nb_of_steps(L):
    nb_of_steps = 1
    if len(L) == 2:
        return 3
    half_size, size = 1, 2
    while half_size < len(L):
        for group in range(len(L) // size):
            top = group * size
            for i in range(half_size):
                if L[top + i] > L[top + i + half_size]:
                    nb_of_steps += 2
                    L[top + i], L[top + i + half_size] =\
                                            L[top + i + half_size], L[top + i]
                else:
                    nb_of_steps += 1
                    if half_size * 2 == len(L) and top == 0 and i == 0\
                       or half_size * 2 == len(L) and top == 0\
                          and i + 1 == half_size:
                        nb_of_steps += 1
        span, double_span = half_size // 2, half_size
        while span:
            skip = half_size // span
            for group in range(len(L) // double_span):
                if (group + 1) % skip == 0:
                    continue
                top = span + group * double_span;
                for i in range(span):
                    nb_of_steps += 1
                    if L[top + i] > L[top + i + span]:
                        nb_of_steps += 1
                        L[top + i], L[top + i + span] =\
                                                 L[top + i + span], L[top + i]
                    elif half_size * 2 == len(L) and span == 1:
                        nb_of_steps += 1
                            
            span //= 2
            double_span //= 2
        half_size *= 2
        size *= 2
    return nb_of_steps

def trace_batcher_sort(L):
    item_colours = initialise_colouring(L)
    display(L, item_colours)
    yield
    if len(L) == 2:
        item_colours[0]['attrs'] = item_colours[1]['attrs'] = 'bold'
        if L[0] > L[1]:
            item_colours[0]['color'] = item_colours[1]['color'] = 'red'
            L[0], L[1] = L[1], L[0]
        else:
            item_colours[0]['color'] = item_colours[1]['color'] = 'yellow'
        display(L, item_colours)
        yield
        item_colours[0]['color'] = item_colours[1]['color'] = 'green'
        item_colours[0]['attrs'] = item_colours[1]['attrs'] = ''
        display(L, item_colours)
        yield
        return
    half_size, size = 1, 2
    while half_size < len(L):
        for group in range(len(L) // size):
            top = group * size
            for i in range(half_size):
                item_colours[top + i]['attrs'] =\
                           item_colours[top + i + half_size]['attrs'] = 'bold'
                if L[top + i] > L[top + i + half_size]:
                    if half_size * 2 == len(L) and top == 0 and i == 0:
                        item_colours[top + i]['color'] =\
                            item_colours[top + i + half_size]['color'] = 'red'
                        display(L, item_colours)
                        yield
                        L[top + i], L[top + i + half_size] =\
                                            L[top + i + half_size], L[top + i]
                        item_colours[top + i]['attrs'] =\
                           item_colours[top + i + half_size]['attrs'] = ''
                        item_colours[top + i]['color'] = 'green'
                        item_colours[top + i + half_size]['color'] = 'yellow'
                        display(L, item_colours)
                        yield
                        item_colours[top + i + half_size]['color'] = 'blue'
                    elif half_size * 2 == len(L) and top == 0\
                         and i + 1 == half_size:
                        item_colours[top + i]['color'] =\
                            item_colours[top + i + half_size]['color'] = 'red'
                        display(L, item_colours)
                        yield
                        L[top + i], L[top + i + half_size]=\
                                            L[top + i + half_size], L[top + i]
                        item_colours[top + i]['attrs'] =\
                           item_colours[top + i + half_size]['attrs'] = ''
                        item_colours[top + i]['color'] = 'yellow'
                        item_colours[top + i + half_size]['color'] = 'green'
                        display(L, item_colours)
                        yield
                        item_colours[top + i]['color'] = 'blue'
                    else:
                        item_colours[top + i]['color'] =\
                            item_colours[top + i + half_size]['color'] = 'red'
                        display(L, item_colours)
                        yield
                        L[top + i], L[top + i + half_size] =\
                                            L[top + i + half_size], L[top + i]
                        item_colours[top + i]['attrs'] =\
                           item_colours[top + i + half_size]['attrs'] = ''
                        item_colours[top + i]['color'] =\
                          item_colours[top + i + half_size]['color'] = 'yellow'
                        display(L, item_colours)
                        yield
                        item_colours[top + i]['color'] =\
                          item_colours[top + i + half_size]['color'] = 'blue'
                else:
                    if half_size * 2 == len(L) and top == 0 and i == 0:
                        item_colours[top + i]['color'] =\
                          item_colours[top + i + half_size]['color'] = 'yellow'
                        display(L, item_colours)
                        yield
                        item_colours[top + i]['attrs'] =\
                          item_colours[top + i + half_size]['attrs'] = ''
                        item_colours[top + i]['color'] = 'green'
                        item_colours[top + i + half_size]['color'] = 'blue'
                        display(L, item_colours)
                        yield
                    elif half_size * 2 == len(L) and top == 0\
                         and i + 1 == half_size:
                        item_colours[top + i]['color'] =\
                          item_colours[top + i + half_size]['color'] = 'yellow'
                        display(L, item_colours)
                        yield
                        item_colours[top + i]['attrs'] =\
                          item_colours[top + i + half_size]['attrs'] = ''
                        item_colours[top + i]['color'] = 'blue'
                        item_colours[top + i + half_size]['color'] = 'green'
                        display(L, item_colours)
                        yield
                    else:
                        item_colours[top + i]['color'] =\
                          item_colours[top + i + half_size]['color'] = 'yellow'
                        display(L, item_colours)
                        yield
                        item_colours[top + i]['attrs'] =\
                        item_colours[top + i + half_size]['attrs'] = ''
                        item_colours[top + i]['color'] =\
                          item_colours[top + i + half_size]['color'] = 'blue'
        span, double_span = half_size // 2, half_size
        while span:
            skip = half_size // span
            for group in range(len(L) // double_span):
                if (group + 1) % skip == 0:
                    continue
                top = span + group * double_span;
                for i in range(span):
                    item_colours[top + i]['attrs'] =\
                             item_colours[top + i + span]['attrs'] = 'bold'
                    if L[top + i] > L[top + i + span]:
                        item_colours[top + i]['color'] =\
                                 item_colours[top + i + span]['color'] = 'red'
                        display(L, item_colours)
                        yield
                        L[top + i], L[top + i + span] =\
                                                 L[top + i + span], L[top + i]
                        item_colours[top + i]['attrs'] =\
                                 item_colours[top + i + span]['attrs'] = ''
                        if half_size * 2 == len(L) and span == 1:
                            item_colours[top + i]['color'] =\
                               item_colours[top + i + span]['color'] = 'green'
                            display(L, item_colours)
                            yield
                        else:
                            item_colours[top + i]['color'] =\
                              item_colours[top + i + span]['color'] = 'yellow'
                            display(L, item_colours)
                            yield
                            item_colours[top + i]['color'] =\
                              item_colours[top + i + span]['color'] = 'blue'
                    else:
                        item_colours[top + i]['color'] =\
                          item_colours[top + i + span]['color'] = 'yellow'
                        display(L, item_colours)
                        yield
                        item_colours[top + i]['attrs'] =\
                                    item_colours[top + i + span]['attrs'] = ''
                        if half_size * 2 == len(L) and span == 1:
                            item_colours[top + i]['color'] =\
                              item_colours[top + i + span]['color'] = 'green'
                            display(L, item_colours)
                            yield
                        else:
                            item_colours[top + i]['color'] =\
                              item_colours[top + i + span]['color'] = 'blue'
            span //= 2
            double_span //= 2
        half_size *= 2
        size *= 2

In [ ]:
widgets.VBox([TraceSorting(trace_batcher_sort, batcher_sort_nb_of_steps,
                           True
                          ).tabs
             ]
             
            )

## Number of comparisons in all cases

`h` takes all values between 1 and $\log_2(N)$. For each $h\in\{1,\dots,\log_2(N)\}$, there are $\frac{N}{2^h}$ associated sorting networks of size $2^h$, each of which performs a number of comparisons $s_h$ that satisfies the recurrence relations that follow.

* $s_1 = 0$
* for all $i\in\{1,\dots,\log_2(N)-1\}$, $s_{i+1}=2s_i+2^h-1$.

It is easily verified by induction that for all $h\in\{1,\dots,\log_2(N)\}$,
$s_h=1+(h-1)2^{h-1}$:

* $1+(0-1)2^{1-1}=1$;
* for all $h\in\{1,\dots,\log_2(N)-1\}$, $2\bigl(1+(h-1)2^{h-1}\bigr)+2^h-1=1+(h-1)2^h+2^h=1+h2^h$.

Hence Batcher sort performs a number of comparisons equal to

\begin{equation*}
\Sigma_{h=1}^{\log_2(N)}\frac{N}{2^h}\bigl(1+(h-1)2^{h-1}\bigr),
\end{equation*}

which simplifies to

\begin{equation*}
N\Bigl(\Sigma_{h=1}^{\log_2(N)}\frac{1}{2^h}+\Sigma_{h=1}^{\log_2(N)}\frac{h-1}{2}\Bigr),
\end{equation*}

which is equal to

\begin{equation*}
N\Bigl(1-\frac{1}{2^{\log_2(N)}}+\frac{\bigl(1+\log_2(N)\bigr)\log_2(N)}{4}-
\frac{\log_2(N)}{2}\Bigr),
\end{equation*}

hence to

\begin{equation*}
N\Bigl(1-\frac{1}{N}+\frac{\bigl(\log_2(N)-1\bigr)\log_2(N)}{4}\Bigr),
\end{equation*}

of complexity $O\bigl(N\log^2(N)\bigr)$.

# Two general complexity results

## Adjacent comparisons

Let $L^{-1}$ denote reversed $L$. For instance, if $N=4$ and $L$ is $[4, 1, 3, 2]$ then $L^{-1}$ is $[2, 3, 1, 4]$. Each of the $\frac{N(N+1)}{2}$ pairs of distinct integers in $\{1,\dots,N\}$ is correctly ordered in $L$ and incorrectly ordered in $L^{-1}$ or the other way around. With this example:

* $(1, 2)$ is correctly ordered in $L$ and incorrectly ordered in $L^{-1}$;
* $(1, 3)$ is correctly ordered in $L$ and incorrectly ordered in $L^{-1}$;
* $(1, 4)$ is correctly ordered in $L^{-1}$ and incorrectly ordered in $L$;
* $(2, 3)$ is correctly ordered in $L^{-1}$ and incorrectly ordered in $L$;
* $(2, 4)$ is correctly ordered in $L^{-1}$ and incorrectly ordered in $L$;
* $(3, 4)$ is correctly ordered in $L^{-1}$ and incorrectly ordered in $L$.

Bubble sort, Selection sort and Insertion only compare values that are adjacent in the list being sorted, swapping them when the values happen to be misordered, thereby decreasing the number of misordered pairs by 1; so when sorting both $L$ and $L^{-1}$, they perform $\frac{N(N+1)}{2}$ swaps of values altogether. Hence any sorting algorithm that only compares values that are adjacent in the list being sorted needs to perform on average at least $\frac{N(N+1)}{2}$ comparisons of values, so an average number of comparisons of values that is of complexity at least $O(N^2)$. In this sense, Bubble sort, Selection sort and Insertion are all optimal in the class of sorting algorithms that perform nothing but adjacent comparisons.

## Arbitrary comparisons

Eight algorithms have been described, implemented, simulated and analysed; all are based on comparing pairs of values. For each comparison, the result, True or False, lets the algorithm deterministically take one of two paths. That makes it possible to map any algorithm of this kind to a decision tree. To make the point more easily, take $N=3$, and write the list $L$ to sort as $[x_1,x_2,x_3]$. An algorithm designed to sort $L$ based on comparing pairs of values can be mapped to a tree $T$ each of whose inner nodes is labelled with a test of the form $x_i<x_j$, $i,j\in\{1,2,3\}$, with a left branch to follow in case the test evaluables to True, and a right branch to follow in case the test evaluates to False. Leaves are labelled with a conclusion, in the form of a permutation of $(x_1,x_2,x_3)$, showing what $L$ eventually becomes thanks to swaps determined by the results of the tests that label the nodes of the path from the root to the leaf. The tree below is what both Bubble sort and Insertion sort map to. The first test performed by both algorithms is whether $x_1<x_2$.

* If the answer is Yes, then no swap is performed and the next test is whether $x_2<x_3$.
    * If the answer is Yes too, then still no swap is performed and both algorithms conclude that the sorted list is $[x_1,x_2,x_3]$ itself.
    * If the answer is False, then $x_1$ and $x_3$ are swapped, so $L$ becomes $[x_1,x_3,x_2]$. A third test is performed, asking whether $x_1<x_3$.
        * If the answer is Yes then no more swap is performed and both algorithms conclude that the sorted list is $[x_1,x_3,x_2]$.
        * If the answer is True then $x_1$ and $x_3$ are swapped and both algorithms conclude that the sorted list is $[x_3,x_1,x_2]$.
* ...

There is one path for each of the 3! permutations of $(x_1,x_2,x_3)$. Both algorithms are such that a swap is performed between $x_i$ and $x_j$, $i,j\in\{1,2,3\}$, after a test $x_i<x_j$ that evaluates to False. This is a special case. In full generality, which swaps are performed, when a swap is performed, is not captured by the tree: we know this from our full knowledge of how the algorithms work. The tree provides only partial information on the algorithms that map to it.

<div><img src="Illustrations/sorting_tree_1.pdf" width="400"/></div>

The tree below is what Selection sort maps to. It has nothing but branches of length 3, consistently with our observation that the algorithm performs 3 comparisons in all cases. Both tests whether $x_2<x_1$ come after the test whether $x_1<x_2$, so have a unique possible outcome determined by the result of the latter. Taking the rightmost branch as an example, we know that there is a fist swap between $x_1$ and $x_3$ after the tests $x_1<x_2$ and $x_2<x_3$ have both been performed and evaluated to False, and there is a second swap between $x_1$ and $x_2$ after the test $x_2<x_1$ has been performed and necessarily evaluated to False.

<div><img src="Illustrations/sorting_tree_2.pdf" width="550"/></div>

For two further examples, here is what Quick sort maps to.

<div><img src="Illustrations/sorting_tree_3.pdf" width="430"/></div>

And here is what Merge sort maps to.

<div><img src="Illustrations/sorting_tree_4.pdf" width="400"/></div>

Let us get back to the general case, so fix $N$ to some arbitrary value, and write $L$ as $[x_1,x_2,\dots,x_N]$. Consider the decision tree that a given algorithm designed to sort $L$ based on comparing pairs of values is mapped to. Since there are $N!$ permutations of $N$ elements, $T$ must have at least $N!$ leaves. If we can verify that the average length of $T$'s paths is at least equal to $\log_2(N!)$, then we can infer that the average length of $T$'s paths is at least equal to $\log_2\bigl(\ulcorner\frac{N}{2}\urcorner^{\ulcorner\frac{N}{2}\urcorner}\bigr)$, equal to $\ulcorner\frac{N}{2}\urcorner\log_2(\ulcorner\frac{N}{2}\urcorner)$, from which we conclude that the algorithm under consideration must perform an average number of comparisons of complexity at least equal to $O\bigl(N\log(N)\bigr)$. In this sense, Merge sort, Quick sort and Heap sort are optimal, whereas Shell sort and Batcher sort aren't.

Let us verify more generally that given a nonzero natural number $n$ and a binary tree $T$ with $n$ leaves, the average length of $T$'s paths is at least equal to $\log_2(n)$. Proof is by induction. Trivially, a binary tree with a unique node has a unique path of length of 1, and $1>\log_2(1)$. Let $n\geq 2$ be given and assume that for all $i\in\{1,\dots,n-1\}$, the average path length of a binary tree with $i$ leaves is at least equal to $\log_2(i)$. Let $T$ be a binary tree with $n$ leaves.

* First assume that $T$'s root has a unique child, and let $T'$ be the subtree of $T$ that is rooted at $T$'s child. By inductive hypothesis, the average length of $T'$'s paths is at least equal to $\log_2(n-1)$. Hence the average length of $T$'s paths is at least equal to $\log_2(n-1)+1$, which is greater than $\log_2(n)$.
* Second, assume that $T$'s root has two children. Let $T_1$ the subtree of $T$ that is rooted at $T$'s left child, and let $T_2$ the subtree of $T$ that is rooted at $T$'s right child. Let $n_1$ and $n_2$ be the number of leaves in $T_1$ and $T_2$, respectively; so $n=n_1+n_2$. Using the inductive hypotheses on $n_1$ and $n_2$, we have that the average length of $T$'s paths is at least equal to

\begin{equation*}
\frac{n_1}{n}\log_2(n_1)+\frac{n_2}{n}\log_2(n_2)+1,
\end{equation*}

and it suffices to show that that expression is at least equal to $\log_2(n)$, which amounts to proving that

\begin{equation*}
\frac{n_1}{n}\log_2(n_1)-\frac{n_1}{n}\log_2(n)+\frac{n_2}{n}\log_2(n_2)-\frac{n_2}{n}\log_2(n)\geq -1,
\end{equation*}

hence that

\begin{equation*}
\frac{n_1}{n}\log_2\bigl(\frac{n_1}{n}\bigr)+\frac{n_2}{n}\log_2\bigl(\frac{n_2}{n}\bigr)\geq -1.
\end{equation*}

For that, it suffices to establish that for all $x\in(0,1)$,

\begin{equation*}
x\log_2(x)+(1-x)\log_2(1-x)\geq -1.
\end{equation*}

But the derivative of $x\log_2(x)+(1-x)\log_2(1-x)$ is $\log_2(x)+\frac{x}{x}-\log_2(1-x)-\frac{1-x}{1-x}$, equal to $\log_2(x)-\log_2(1-x)$, which is strictly negative when $x\in(0,\frac{1}{2})$, equal to 0 when $x=\frac{1}{2}$, and strictly positive when $x\in(\frac{1}{2},1)$; moreover, $\frac{1}{2}\log_2(\frac{1}{2})+(1-\frac{1}{2})\log_2(1-\frac{1}{2})=-\frac{1}{2}-\frac{1}{2}=-1$, and we are done.